In [ ]:
import sys, os
import serial
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import win32com.client
from astropy.io import fits
import threading
import nbimporter
from picoammeter_control import (
    send_cmd,
    query,
    init_cmds
)
from camera_control_ascom import (
    take_ramp,
    camera_init,
    populate_header
)
sys.path.insert(0, os.path.abspath('.'))
sys.path.insert(0, os.path.abspath('filter_wheel_control'))
from filter_wheel_control.wheel_controller import (
    list_devices,
    open_device,
    get_position,
    get_position_count,
    set_position,
    close_device
)
from data_helper_functions import (
    init_wheels,
    close_wheels,
    camera_init,
    picoam_stability,
    picoam_worker_exp,
    camera_with_picoam,
    collect_var_exp,
    var_exp_graphs,
    capture_frame,
    calculate_gain,
    get_gain,
    picoam_worker_filters,
    camera_with_filters,
    collect_var_filters,
    compare_exp_and_filter,
    collect_bias_frames
)

In [ ]:
camera_init()
cam_name = "ASCOM.ASICamera2.Camera"
save_path= "C:/Users/Jonah/fits_imgs"

In [ ]:
min_exp = 0.00025
max_exp = 6.0

In [ ]:
init_wheels(print_stuff=False)
close_wheels()


In [ ]:
e_per_adu = get_gain(cam_name=cam_name, exp_time=min_exp, num_frames=5, print_stuff=True)

In [ ]:
I_flat, Q_ramp, mean_signal, std_signal, bit_depth, exp_times, exp_low, exp_high, exp_50 = (
    collect_var_exp(cam_name=cam_name, num_exp=20, num_frames=3, exp_low = min_exp, exp_high = max_exp, save_path=save_path, offset=0)
)

# var_exp_graphs(mean_signal, std_signal, bit_depth, exp_times, exp_low, exp_high, save=True, e_per_adu=e_per_adu, min_percent=0.001, percent_change=True)

In [ ]:
signals_f, Q_filt = collect_var_filters(save_path = save_path, cam_name=cam_name, exp_50=exp_50, num_frames=3, offset=0)

In [ ]:
bias_level = collect_bias_frames(cam_name = cam_name,num_frames=10, exp_low=min_exp, save_path=save_path, offset=0)
print("bias level: ", bias_level)


In [ ]:
compare_exp_and_filter(e_per_adu = .025, Q_ramp=Q_ramp, Q_filt=Q_filt, signals_f=signals_f, mean_signal=mean_signal, bias_level=bias_level, save=True)
